In [1]:
import pandas as pd
import numpy as np

wdata = open('datasets/web_log_data.csv','r').readlines()

print('\n'.join(wdata[:3]))

ip,date_time,request,step,session,user_id

c210-49-32-6.rochd2.,18/Apr/2005:21:25:07,/,1,3,3

visp.inabox.telstra.,19/Apr/2005:08:24:28,/,1,12,12



In [2]:
# set names of pandas dataframe
names=['Host','Datetime', 'Request', 'Step', 'Session', 'User_ID']
# read the dataframe
df = pd.read_csv('datasets/web_log_data.csv', sep=',', names=names, header=None)

In [3]:
df.head()

,Host,Datetime,Request,Step,Session,User_ID
0,ip,date_time,request,step,session,user_id
1,c210-49-32-6.rochd2.,18/Apr/2005:21:25:07,/,1,3,3
2,visp.inabox.telstra.,19/Apr/2005:08:24:28,/,1,12,12
3,dsl-61-95-54-84.requ,19/Apr/2005:08:33:01,/,1,13,13
4,d220-236-91-52.dsl.n,19/Apr/2005:09:16:06,/,1,15,15


In [7]:
mask = (df['Request'].str.endswith('.css') | df['Request'].str.endswith('.ico') | df['Request'].str.endswith('.js') | df['Request'].str.endswith('.txt'))
print("# Rows before:", len(df))

# invert the mask, only keep records without the wrong extensions in the request column
df2 = df[~mask]

print("After images removal", len(df2))
df2.info()

# Rows before: 5867
After images removal 4459
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4459 entries, 0 to 5866
Data columns (total 6 columns):
Host        4459 non-null object
Datetime    4459 non-null object
Request     4459 non-null object
Step        4459 non-null object
Session     4459 non-null object
User_ID     4459 non-null object
dtypes: object(6)
memory usage: 243.9+ KB


In [ ]:
#This doesn't seem to work
#for hosts in df2['Host']:
 #   if hosts.rfind('/') != 0:
 #       hosts + '/'

        
